In [2]:
import numpy as np
import matplotlib.pyplot as plt
from winsound import Beep
from datetime import datetime
from multiprocessing import cpu_count, Pool
import  multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor

def qdot(state, params):
    return state[:, 2:4]


def pdot(state, params):
    positions = state[:, 0:2]
    L = params[0]  #length scale of potential
    A = params[1]  #Strength of potential
    rij = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]
    #shift as needed
    mask = np.abs(rij[:,:,0])>0.5
    rij[mask,0] = rij[mask,0] - np.sign(rij[mask,0])
    mask = np.abs(rij[:,:,1])>0.5
    rij[mask,1] = rij[mask,1] - np.sign(rij[mask,1])
    dsq = np.linalg.norm(rij, axis=2)**2
    dsq = np.stack([dsq, dsq], axis=2)
    force = 2*A*rij/(L**2)*np.exp(-dsq/(L**2))
    v = np.sum(force, axis=0)

    return v


def fix_state(state):
    N = np.size(state, 0)
    for i in np.arange(0, N, 1):
        for j in np.arange(0, 2, 1):
            if state[i, j] > 1:
                state[i, j] = state[i, j] - 1
            if state[i, j] < 0:
                state[i, j] = state[i, j] + 1
    return state


def symplectic_step(state, h, params):
    #Third order symplectic step
    #Taken from Wikipedia 
    state[:, 0:2] = state[:, 0:2] + 1 * h * qdot(state, params)
    fix_state(state)
    state[:, 2:4] = state[:, 2:4] - 1 / 24 * h * pdot(state, params)
    state[:, 0:2] = state[:, 0:2] - 2 / 3 * h * qdot(state, params)
    fix_state(state)
    state[:, 2:4] = state[:, 2:4] + 3 / 4 * h * pdot(state, params)
    state[:, 0:2] = state[:, 0:2] + 2 / 3 * h * qdot(state, params)
    fix_state(state)
    state[:, 2:4] = state[:, 2:4] + 7 / 24 * h * pdot(state, params)

    return state

def save_plot(args):
    i, state = args
    #scatter plot state
    fig = plt.figure(dpi=400)
    plt.scatter( state[:,0], state[:,1] )
    plt.xlim([0,1])
    plt.ylim([0,1])
    fig.savefig('gas/' + str(i) + '.png')
    plt.close()

In [3]:
%%prun

st = datetime.now()
N = 64  #number of particles

L = 1 / 10  #interaction length
A = 1  #interaction strength
params = np.array([L, A])

every = 4  #take this many timesteps before saving to array
M = 128  #saved timesteps

h = 1e-2  #the true timestep used will be h/every

traj = np.zeros([N, 4, M])
state = np.zeros([N, 4])

#initial data
#state[i,:] = (xi,yi,vxi,vyi)
state[:, 0:2] = np.random.rand(N, 2)
#state[:,2] = np.sin( 4*np.pi*state[:,1] ) + 0.1*np.sin( 6*np.pi*state[:,1] - 2 )
#state[:,3] = np.sin( 4*np.pi*state[:,0] )

traj[:, :, 0] = state

for i in np.arange(1, M, 1):
    for j in np.arange(0, every, 1):
        state = symplectic_step(state, h / every, params)
    traj[:, :, i] = state
print(datetime.now() - st)
st = datetime.now()


def iterate_array(arr):
    for i in range(arr.shape[-1]):
        yield i, arr[:,:, i]


for e in iterate_array(traj):
    save_plot(e)


print(datetime.now() - st)

Beep(500, 500)

0:00:01.650997
 